In [327]:
import math

class ConvexHull:
    def __init__(self, points):
        self.hull = self.convex_hull_2d(points)
    
    def convex_hull_2d(self, points):
        if len(points) <= 5:
            return self.find_convex_hull_graham(points)
        else:
            midpoint = int(len(points)/2)
#             print(midpoint)
            left_hull = self.convex_hull_2d(points[:midpoint])
            right_hull = self.convex_hull_2d(points[midpoint:])
            print(left_hull, right_hull)
        return self.merge_hull(left_hull, right_hull)


    def sort_points_ccw(self,points):
        center = tuple(map(sum, zip(*points)))  # Find the centroid of the points
        angles = []
        for point in points:
            x, y = point
            angle = math.atan2(y - center[1], x - center[0])
            angles.append((angle, point))
        angles.sort()
        return [point for angle, point in angles]

    import math

    from math import atan2

    def polar_angle(self, p0, p1):
        x, y = p1[0] - p0[0], p1[1] - p0[1]
        return math.atan2(y, x)
    
    def sort_points(self, reference_point, points):
        return sorted(points, key=lambda point: self.polar_angle(reference_point, point))
        

    def find_convex_hull_graham(self, points):
#         print(points)
        # Find the point with the smallest y-coordinate (ties are broken by the one with the smaller x-coordinate)
        reference_point = min(points, key=lambda point: (point[1], point[0]))

        # Sort the points based on their polar angle with the reference point
        sorted_points = sorted(points, key=lambda point: self.polar_angle(reference_point, point))

        # Initialize the stack with the first three points
        stack = [sorted_points[0], sorted_points[1], sorted_points[2]]

        # Iterate over the rest of the points
        for point in sorted_points[3:]:
            # Remove any points from the stack that create a concave angle with the current point
            while len(stack) >= 2 and abs(self.polar_angle(stack[-2], stack[-1])) >= abs(self.polar_angle(stack[-1], point)):
                stack.pop()

            # Add the current point to the stack
            stack.append(point)

        return stack
    
    def y(self, i, j, left_hull, right_hull):
        p1 = left_hull[i]
        p2 = right_hull[j]
        m = (p2[1] - p1[1]) / (p2[0] - p1[0])
        b = p1[1] - m * p1[0]
        return m * (self.middle) + b
    
    def find_upper_tangent(self, i, j, left_hull, right_hull):
        # Find the y-intercept of the line between the rightmost point of the left hull
        # and the leftmost point of the right hull
        p,q = len(left_hull),len(right_hull)
        #define MIDDLE
        
        s = self.y(i,j, left_hull, right_hull)
    #     print(left_hull[i], right_hull[j],i,j,s,y(i,(j+1)%q,left_hull,right_hull), y((i-1)%p,j,left_hull,right_hull))

        while self.y(i,(j-1)%q,left_hull,right_hull) > s or self.y((i+1)%p,j,left_hull,right_hull) > s:
    #         print(left_hull[i], right_hull[j],i,j,s,y(i,(j+1)%q,left_hull,right_hull), y((i-1)%p,j,left_hull,right_hull))
            if self.y(i, (j-1)%q, left_hull, right_hull) > s:
                j -= 1
                j %= q
                s = self.y(i, j, left_hull, right_hull)
            elif self.y((i+1)%p, j, left_hull, right_hull) > s:
                i += 1
                i %= p
                s = self.y(i, j, left_hull, right_hull)
    #     print(i,j,s,y(i,(j+1)%q,left_hull,right_hull), y((i-1)%p,j,left_hull,right_hull))

        return (i, j)
    
    def find_lower_tangent(self,i, j, left_hull, right_hull):
        # Find the y-intercept of the line between the rightmost point of the left hull
        # and the leftmost point of the right hull
        p,q = len(left_hull),len(right_hull)
        s = self.y(i,j, left_hull, right_hull)

        while self.y(i,(j+1)%q,left_hull,right_hull) < s or self.y((i-1)%p,j,left_hull,right_hull) < s:
            if self.y((i-1)%p, j, left_hull, right_hull) < s:
                i -= 1
                i %= len(left_hull)
                s = y(i, j, left_hull, right_hull)
            elif self.y(i, (j+1)%q, left_hull, right_hull) < s:
                j += 1
                j %= q
                s = y(i, j, left_hull, right_hull)

        return (i, j)
    
    def merge_lists(self, lower_hull, upper_hull, lower_tangent, upper_tangent):
        # Find the indices of the points corresponding to the lower and upper tangents
#         print(lower_tangent, upper_tangent)
        i = lower_hull.index(lower_tangent[0])
        j = lower_hull.index(upper_tangent[0])

        ii = upper_hull.index(lower_tangent[1])
        jj = upper_hull.index(upper_tangent[1])

        # Remove the points between the lower and upper tangents, if any
        print(i,j, lower_hull)
        print(ii,jj,upper_hull)
        merged_hull = []
        
        while i != j:
            merged_hull.append(lower_hull[i])
            i += 1
            i %= len(lower_hull)
        merged_hull.append(lower_hull[j])
        
        while jj != ii:
            merged_hull.append(upper_hull[jj])
            jj += 1
            jj %= len(upper_hull)
        merged_hull.append(upper_hull[jj])
            
        return merged_hull
    
    def merge_hull(self, left_hull, right_hull):
        # sort left_hull counterclockwise and right_hull clockwise based on center point
        left_center = [sum(p[0] for p in left_hull)/len(left_hull), sum(p[1] for p in left_hull)/len(left_hull)]
        left_hull = sorted(left_hull, key=lambda p: (math.atan2(p[1]-left_center[1], p[0]-left_center[0]), -p[1]))
        right_center = [sum(p[0] for p in right_hull)/len(right_hull), sum(p[1] for p in right_hull)/len(right_hull)]
        right_hull = sorted(right_hull, key=lambda p: (math.atan2(p[1]-right_center[1], p[0]-right_center[0]), p[1]))

        # find leftmost and rightmost points
        rightmost = max(left_hull, key=lambda p: p[0])
        self.middle = rightmost[0]
        rightmost = left_hull.index(rightmost)
        leftmost = min(right_hull, key=lambda p: p[0])
        self.middle += leftmost[0]
        leftmost = right_hull.index(leftmost)
        self.middle /= 2

        # find upper and lower tangents

        i, j = self.find_upper_tangent(rightmost, leftmost, left_hull, right_hull)
        ii, jj = self.find_lower_tangent(rightmost, leftmost, left_hull, right_hull)
        return self.merge_lists(left_hull,right_hull, [left_hull[i],right_hull[j]], [left_hull[ii],right_hull[jj]])


In [329]:
if __name__ == '__main__':
    hull = ConvexHull(sorted([(1,2),(4,6),(9,10),(19,10),(17,6),(10,0),(8,19),(7,5),(17,2),(15,1)]))

    print('Convex Hull:')
    for x in hull.hull:
        print(int(x[0]), int(x[1]))

[(1, 2), (7, 5), (9, 10), (8, 19)] [(10, 0), (15, 1), (17, 2), (19, 10)]
3 0 [(1, 2), (7, 5), (9, 10), (8, 19)]
3 0 [(10, 0), (15, 1), (17, 2), (19, 10)]
Convex Hull:
8 19
1 2
10 0
15 1
17 2
19 10
